In [48]:
import pandas as pd

file_to_load = "Resources/purchase_data.csv"

purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [49]:
unique_players = purchase_data["SN"].unique()
player_series = pd.Series(unique_players)
player_num = player_series.count()

player_df = pd.DataFrame({"Total Number of Players": [player_num]})
player_df

,Total Number of Players
0,576


In [50]:
unique_items = purchase_data["Item ID"].unique()
unique_series = pd.Series(unique_items)
item_num = unique_series.count()

avg_buy = purchase_data["Price"].mean()

all_buys = purchase_data["Purchase ID"].count()

rev = purchase_data["Price"].sum()

buying_df = pd.DataFrame(
    {"Number of Unique Items": [item_num],
    "Average Price": [avg_buy],
    "Number of Purchases": [all_buys],
    "Total Revenue": rev})

buying_df["Average Price"] = buying_df["Average Price"].map("${:.2f}".format)
buying_df["Total Revenue"] = buying_df["Total Revenue"].map("${:.2f}".format)

buying_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [51]:
df1 = purchase_data.drop_duplicates(subset=['SN'],keep='first')
length = df1.shape[0]
df1 = pd.DataFrame(df1.groupby('Gender')['SN'].count()).rename(columns={'SN':'amount_gender'})
df1['pct_gender'] = df1['amount_gender'] / length
df1['pct_gender'] = round(df1['pct_gender']*100,2)
df1

,amount_gender,pct_gender
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


In [52]:
df2 = pd.DataFrame(purchase_data.groupby("Gender")["Purchase ID"].count()).rename(columns = {"Purchase ID" : "Purchase Count"})
purchases = purchase_data.groupby("Gender")["Price"].sum()
people = purchase_data.groupby("Gender")["SN"].count()
df2["Average Purchase Price"] = round(purchases/df2["Purchase Count"],2)
df2["Total Purchase Price"] = purchases
df2["Average Total per Person"] = round(purchases/purchase_data.SN.nunique(),2)

df2["Average Purchase Price"] = df2["Average Purchase Price"].map("${:.2f}".format)
df2["Total Purchase Price"] = df2["Total Purchase Price"].map("${:.2f}".format)
df2["Average Total per Person"] = df2["Average Total per Person"].map("${:.2f}".format)
df2

,Purchase Count,Average Purchase Price,Total Purchase Price,Average Total per Person
Gender,,,,
Female,113,$3.20,$361.94,$0.63
Male,652,$3.02,$1967.64,$3.42
Other / Non-Disclosed,15,$3.35,$50.19,$0.09


In [76]:
bins = [0,10,14,19,24,29,34,39,1000]
ages1 = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

dem_df = purchase_data.drop_duplicates(subset = ['SN'], keep = 'first')
dem_df['Age Ranges'] = pd.cut(dem_df['Age'], bins, labels = ages1, include_lowest = True)
length = dem_df.shape[0
                     ]
dem_mid = dem_df.groupby('Age Ranges')
dem_count = dem_mid['SN'].count()
dem_perc = (dem_count/length)*100

dem_fin = pd.DataFrame({"Total Count": dem_count,"Percentage of Players":dem_perc})

dem_fin['Percentage of Players'] = dem_fin['Percentage of Players'].map("{:.2f}%".format)
dem_fin


<ipython-input-76-01b2716eeca7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dem_df['Age Ranges'] = pd.cut(dem_df['Age'], bins, labels = ages1, include_lowest = True)


,Total Count,Percentage of Players
Age Ranges,,
<10,24,4.17%
10-14,15,2.60%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [53]:
bins = [0,10,14,19,24,29,34,39,1000]
ages = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

age_df = purchase_data.copy(deep = True)
age_df['Age Ranges'] = pd.cut(age_df['Age'], bins, labels=ages, include_lowest = True)

age_group = age_df.groupby('Age Ranges')
purchases = age_group["SN"].count()
averages = age_group['Price'].mean()
totalies = age_group['Price'].sum()

age_fin = pd.DataFrame({"Purchase Count": purchases,"Average Purchase Price": averages, "Total":totalies})

age_fin["Average Purchase Price"] = age_fin["Average Purchase Price"].map("${:.2f}".format)
age_fin['Total'] = age_fin['Total'].map("${:.2f}".format)

age_fin

,Purchase Count,Average Purchase Price,Total
Age Ranges,,,
<10,32,$3.40,$108.96
10-14,19,$2.68,$50.95
15-19,136,$3.04,$412.89
20-24,365,$3.05,$1114.06
25-29,101,$2.90,$293.00
30-34,73,$2.93,$214.00
35-39,41,$3.60,$147.67
40+,13,$2.94,$38.24


In [64]:
#Top 5 Spenders by purchase count then list (SN, Purchase Count, Avg Purchase Price, Total Purchase Value)
spender_df = purchase_data.copy(deep = True)
spender_start = spender_df.groupby('SN')

purch_count = spender_start['Item Name'].count()
avg_purch = spender_start['Price'].mean()
total_purch = spender_start['Price'].sum()

spender_mid = pd.DataFrame({"Purchase Count":purch_count,"Average Purchase Price":avg_purch,"Total Purchase Value":total_purch})

spender_fin = spender_mid.sort_values("Total Purchase Value", ascending = False)

spender_fin["Average Purchase Price"] = spender_fin["Average Purchase Price"].map("${:.2f}".format)
spender_fin['Total Purchase Value'] = spender_fin['Total Purchase Value'].map("${:.2f}".format)

spender_fin.iloc[:5]





,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [69]:
item_df = purchase_data.copy(deep=True)
item_start = item_df.groupby('Item ID')

item_count = item_start['SN'].count()
item_name = item_start['Item Name'].unique()
item_price = item_start['Price'].mean()
item_rev = item_start['Price'].sum()

item_mid = pd.DataFrame({"Item Name": item_name,"Purchase Count":item_count,"Item Price":item_price,"Total Purchase Value":item_rev})

item_fin = item_mid.sort_values('Purchase Count', ascending = False)

item_fin["Item Price"] = item_fin["Item Price"].map("${:.2f}".format)
item_fin['Total Purchase Value'] = item_fin['Total Purchase Value'].map("${:.2f}".format)

item_fin.iloc[:5]

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,[Final Critic],13,$4.61,$59.99
178,"[Oathbreaker, Last Hope of the Breaking Storm]",12,$4.23,$50.76
145,[Fiery Glass Crusader],9,$4.58,$41.22
132,[Persuasion],9,$3.22,$28.99
108,"[Extraction, Quickblade Of Trembling Hands]",9,$3.53,$31.77


In [70]:
profit_df = purchase_data.copy(deep=True)
profit_start = profit_df.groupby('Item ID')

profit_count = profit_start['SN'].count()
profit_name = profit_start['Item Name'].unique()
profit_price = profit_start['Price'].mean()
profit_rev = profit_start['Price'].sum()

profit_mid = pd.DataFrame({"Item Name": profit_name,"Purchase Count":profit_count,"Item Price":profit_price,"Total Purchase Value":profit_rev})

profit_fin = profit_mid.sort_values('Total Purchase Value', ascending = False)

profit_fin["Item Price"] = profit_fin["Item Price"].map("${:.2f}".format)
profit_fin['Total Purchase Value'] = profit_fin['Total Purchase Value'].map("${:.2f}".format)

profit_fin.iloc[:5]

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,[Final Critic],13,$4.61,$59.99
178,"[Oathbreaker, Last Hope of the Breaking Storm]",12,$4.23,$50.76
82,[Nirvana],9,$4.90,$44.10
145,[Fiery Glass Crusader],9,$4.58,$41.22
103,[Singed Scalpel],8,$4.35,$34.80
